In [2]:
from framework.Platform import PlatformManager
from framework.Configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=True,
    fid=True,
    kid=True
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

platform_manager.calc_metrics()



Real images found, name:CelebA_Original
3 different generator sources found. Names:
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP
[START]: Calculating Metrics for diffusionStyleGAN2


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Processing samples                                                               
Extracting features from input2
Processing samples                                                             
Inception Score: 3.5856323974726556 ± 0.020385455507128
Frechet Inception Distance: 11.367448876552714
Kernel Inception Distance: 0.01063310997747744 ± 0.0006198271780372778          
Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1


[FINISHED]: Calculating Metrics for diffusionStyleGAN2
{'Inception Score Mean': 3.5856323974726556, 'Inception Score Std': 0.020385455507128, 'Frechet Inception Distance': 11.367448876552714, 'Kernel Inception Distance Mean': 0.01063310997747744, 'Kernel Inception Distance Std': 0.01063310997747744}
[START]: Calculating Metrics for diffusionVAE


Processing samples                                                               
Extracting features from input2
Processing samples                                                             
Inception Score: 3.5856323974726556 ± 0.020385455507128
Frechet Inception Distance: 16.49486166225401
Kernel Inception Distance: 0.016086738425925923 ± 0.0007564590874885127         
Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']


[FINISHED]: Calculating Metrics for diffusionVAE
{'Inception Score Mean': 3.5856323974726556, 'Inception Score Std': 0.020385455507128, 'Frechet Inception Distance': 16.49486166225401, 'Kernel Inception Distance Mean': 0.016086738425925923, 'Kernel Inception Distance Std': 0.016086738425925923}
[START]: Calculating Metrics for guidedDiffusion_IP


Extracting features from input1
Processing samples                                                               
Extracting features from input2
Processing samples                                                             
Inception Score: 3.5856323974726556 ± 0.020385455507128
Frechet Inception Distance: 7.617265591198162
                                                                                

[FINISHED]: Calculating Metrics for guidedDiffusion_IP
{'Inception Score Mean': 3.5856323974726556, 'Inception Score Std': 0.020385455507128, 'Frechet Inception Distance': 7.617265591198162, 'Kernel Inception Distance Mean': 0.008803351719219222, 'Kernel Inception Distance Std': 0.008803351719219222}


Kernel Inception Distance: 0.008803351719219222 ± 0.0006384332745241464


In [2]:
import torch_fidelity
import torch

print(torch.cuda.is_available())

metrics_dict = torch_fidelity.calculate_metrics(
    input1=platform_manager.helper.real_images_src.get_dataset(), 
    input2=platform_manager.helper.get_generator_src("guidedDiffusion_IP").get_dataset(), 
    cuda=True, 
    isc=True, 
    fid=True, 
    kid=True, 
    prc=True, 
    verbose=True,
    save_cpu_ram=True,
)

True


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased', '2048']
Extracting features from input1
Processing samples                                                               
Extracting features from input2
Processing samples                                                             
Inception Score: 3.5856323974726556 ± 0.020385455507128
Frechet Inception Distance: 7.617265591198162
Kernel Inception Distance: 0.008803351719219222 ± 0.0006384332745241464         
Creating feature extractor "vgg16" with features ['fc2_relu']
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\qrno9/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:21<00:00, 25.6MB/s] 
Extracting features from input1
Processing samples                                                               
Extracting features from input2
Processing samples                                                             


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 8103960000 bytes.

In [3]:
metrics_dict

{'inception_score_mean': 3.5856323974726556,
 'inception_score_std': 0.020385455507128,
 'frechet_inception_distance': 7.61726559120001,
 'kernel_inception_distance_mean': 0.008803351719219222,
 'kernel_inception_distance_std': 0.0006384332745241464}

In [1]:
from framework.util import celebA_to_64

celebA_to_64("D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\original_images\images\\",
             "D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\original_64\\")

(202599, 64, 64, 3) size array saved into celeba64_train.npz
